In [0]:
df = spark.read.format("csv").option("inferSchema", True).option("header", True).load("abfss://bronze@netflixdatalakejess.dfs.core.windows.net/netflix_titles")

Came across a major error that affected my major workflow that is due to type inference here so i had to place it on the top for debugging

In [0]:
from pyspark.sql.functions import col

df_clean = df.filter(col("duration_minutes").rlike("^[0-9]+$"))
df_clean = df_clean.withColumn("duration_minutes", col("duration_minutes").try_cast("integer"))
df_clean.display()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df_clean = df_clean.withColumn("duration_minutes", col("duration_minutes").cast("integer")).withColumn("duration_seasons", col("duration_seasons").cast("integer"))

In [0]:
df_clean.display()

In [0]:
df_clean = df_clean.fillna({"duration_minutes": 0, "duration_seasons": 1})

In [0]:
df_clean = df_clean.withColumn("short_title", split(col("title"), ":")[0])

In [0]:
df_clean = df_clean.withColumn("type_flag", when(col("type")=="Movie", 1)\
    .when(col("type")=="TV Show", 2).otherwise("0"))

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

In [0]:

df_clean = df_clean.withColumn("duration_ranking", dense_rank().over(Window.orderBy(desc("duration_minutes"))))

In [0]:
df_clean.display()

In [0]:
df_clean.write.format("delta").mode("overwrite").option("mergeSchema", True).option("path", "abfss://silver@netflixdatalakejess.dfs.core.windows.net/netflix_titles").save()